In [1]:
import pandas as pd
import os

In [2]:
def traiter_donnees_par_annee(annee, repertoire_entree, repertoire_sortie, finess,
                               fichiers_traite, fichiers_non_trouves):
    # Construction des chemins de fichiers
    fichier_rpsa = f"{finess}.{annee}.12.rpsa"
    fichier_isoconta = f"{finess}.{annee}.12.isoconta"
    fichier_tra = f"TRA.{finess}.{annee}.crypte.txt"

    chemin_rpsa = os.path.join(repertoire_entree, fichier_rpsa)
    chemin_isoconta = os.path.join(repertoire_entree, fichier_isoconta)
    chemin_tra = os.path.join(repertoire_sortie, fichier_tra)

    fichiers = {
        "RPSA": chemin_rpsa,
        "ISOCONTA": chemin_isoconta,
        "TRA": chemin_tra
    }

    # Vérification de l'existence des fichiers
    for nom, chemin in fichiers.items():
        if not os.path.exists(chemin):
            fichiers_non_trouves.append(os.path.basename(chemin))
            return  # On saute cette année si un fichier est manquant

    # Lecture des fichiers
    RPSA = pd.read_csv(chemin_rpsa, names=["colRPSA"], dtype=str)
    TRA = pd.read_csv(chemin_tra, names=["colTRA"], dtype=str)
    ISOCONTA = pd.read_csv(chemin_isoconta, names=["colISOCONTA"], dtype=str)

    # Extraction des champs
    RPSA["num_sequ"] = RPSA["colRPSA"].str[98:104]
    

    TRA["IPP"] = TRA["colTRA"].str[0:24]
    TRA["num_sequ"] = TRA["colTRA"].str[43:49]

    ISOCONTA["num_sequ"] = ISOCONTA["colISOCONTA"].str[29:35]

    # Lister les num_sequ du RPSA
    numseq_unique = RPSA["num_sequ"].unique().tolist()

    # Extraction des IPP correspondants
    IPPSE = TRA[TRA["num_sequ"].isin(numseq_unique)]["IPP"].unique().tolist()
    TRA_filtered = TRA[TRA["IPP"].isin(IPPSE)]
    num_sequ_uniques = TRA_filtered["num_sequ"].unique()

    # Mapping IPP -> num_sequ
    tra_unique = TRA.drop_duplicates(subset="num_sequ", keep="first").set_index("num_sequ")["IPP"]

    # RPSA filtré
    RPSA_filtered = RPSA[RPSA["num_sequ"].isin(num_sequ_uniques)].copy()
    RPSA_filtered["IPP"] = RPSA_filtered["num_sequ"].map(tra_unique)
    RPSA_filtered = RPSA_filtered.drop(columns=["num_sequ"])
    nom_fichier_rpsa = os.path.join(repertoire_sortie, f"RPSA.Filtre.{finess}.{annee}.txt")
    RPSA_filtered.to_csv(nom_fichier_rpsa, index=False, header=None)

    # TRA filtré
    TRA_filtered = TRA_filtered.drop(columns=["num_sequ", "IPP"])
    nom_fichier_tra = os.path.join(repertoire_sortie, f"TRA.Filtre.{finess}.{annee}.txt")
    TRA_filtered.to_csv(nom_fichier_tra, index=False, header=None)

    # ISOCONTA filtré
    ISOCONTA_filtered = ISOCONTA[ISOCONTA["num_sequ"].isin(num_sequ_uniques)].copy()
    ISOCONTA_filtered["IPP"] = ISOCONTA_filtered["num_sequ"].map(tra_unique)
    ISOCONTA_filtered = ISOCONTA_filtered.drop(columns=["num_sequ"])
    nom_fichier_isoconta = os.path.join(repertoire_sortie, f"ISOCONTA.Filtre.{finess}.{annee}.txt")
    ISOCONTA_filtered.to_csv(nom_fichier_isoconta, index=False, header=None)

    fichiers_traite.extend([nom_fichier_rpsa, nom_fichier_tra, nom_fichier_isoconta])

In [3]:
def main():
    print("=== Traitement des fichiers RPSA, ISOCONTA et TRA (2021-2024) ===")
    repertoire_entree = input("📥 Répertoire d'entrée : ").strip()
    repertoire_sortie = input("📤 Répertoire de sortie : ").strip()
    finess = input("🏥 Numéro de FINESS : ").strip()

    annees = [2021]
    fichiers_traite = []
    fichiers_non_trouves = []

    for annee in annees:
        traiter_donnees_par_annee(annee, repertoire_entree, repertoire_sortie, finess,
                                  fichiers_traite, fichiers_non_trouves)

    # Résumé final
    print("\n=== RÉSUMÉ DU TRAITEMENT ===")
    if fichiers_traite:
        print("✅ Fichiers traités avec succès :")
        for f in fichiers_traite:
            print("  -", os.path.basename(f))
    if fichiers_non_trouves:
        print("\n⚠️ Fichiers manquants :")
        for f in fichiers_non_trouves:
            print("  -", f)

In [4]:
if __name__ == "__main__":
    main()

=== Traitement des fichiers RPSA, ISOCONTA et TRA (2021-2024) ===


📥 Répertoire d'entrée :  C:\Users\Youba\Documents\
📤 Répertoire de sortie :  C:\Users\Youba\Documents\
🏥 Numéro de FINESS :  200001



=== ✅ RÉSUMÉ DU TRAITEMENT ===

⚠️ Fichiers manquants :
  - 200001.2024.12.rpsa
  - 200001.2023.12.rpsa
  - 200001.2022.12.rpsa
  - 200001.2021.12.rpsa


***Préparé par : Lekehal Hammada***

***Équipe : I4S***

***30/04/2025***